In [26]:
import os
from pathlib import Path
import math
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time

In [2]:
DATA_PATH = Path.cwd().parent.parent / "data" / "raw"

In [3]:
recipes_urls = pd.read_csv(DATA_PATH / "urls.csv")
recipes_urls = recipes_urls['url'].to_list()

## Define function to load data

need to make https://www.veganrecipeclub.org.uk/recipes/15-minute-flourless-choc-chip-banana-muffins/ work

### Find the different section of the web page

In [4]:
#Title
def get_title(soup):
    return soup.find_all("h1")[0].get_text(strip=True)

In [5]:
#section name
def get_sections(soup):
    elements = dict()
    sections = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    for i, section in enumerate(sections):
        elements[section.text] = i
    return elements

In [6]:
#Preparation time
def get_preparation_time(soup, elements):
    time = 0
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

    try:
        if(html_element[elements['Preparation time']].nextSibling.nextSibling.text != None):
            time = html_element[elements['Preparation time']].nextSibling.nextSibling.text
    except:
        return 0

    return time

#Cooking time
def get_cooking_time(soup, elements):
    time = 0
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

    try:
        if(html_element[elements['Cooking time']].nextSibling.nextSibling.text != None):
            time = html_element[elements['Cooking time']].nextSibling.nextSibling.text
    except:
        return 0

    return time

#Preparation time
def get_total_time(soup, elements):
    time = 0
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

    try:
        if(html_element[elements['Total time']].nextSibling.nextSibling.text != None):
            time = html_element[elements['Total time']].nextSibling.nextSibling.text
    except:
        return 0

    return time

In [7]:
#Style of cooking
def get_cooking_style(soup):
    cooking_styles = ''
    styles = soup.find_all("p", {"class": "intList"})
    if len(styles) != 0:
        for style in styles:
            cooking_styles += style.get_text(strip=True) + ' '
    return cooking_styles

In [8]:
#Type of recipe
def get_recipe_type(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    recipe_types = ''
    try:
        types = html_element[elements['Meal']].nextSibling.nextSibling.find_all("li")
        for type_ in types:
            recipe_types += type_[0] + ' '
    except:
        return ''
    return recipe_types

In [9]:
#Number of servings
def get_number_servings(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    try:
        return html_element[elements['Servings']].nextSibling.nextSibling.get_text()
    except:
        return 'unknown'

In [10]:
#Ingredients
def get_ingredients(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    ingredients = []
    try:
        HTML_ingredients = html_element[elements['Ingredients']].nextSibling.nextSibling
        for item in HTML_ingredients.find_all("li"):
            ingredients.append(item.get_text())

        if(len(HTML_ingredients.find_all("li")) == 0):
            #Ingredients
            HTML_ingredients = HTML_ingredients[elements['Ingredients']].parent.find("div", {"class": "recipe-list"}).p
            for ingredient in HTML_ingredients.get_text().split("\n"):
                ingredients.append(ingredient)
    except:
        return []
    return ingredients

In [11]:
#Instructions
def get_instructions(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    instructions = []
    try: 
        HTML_instructions = html_element[elements['Instructions']].nextSibling.nextSibling
        for item in HTML_instructions.find_all("li"):
            instructions.append(item.get_text())
    except:
        return []
    return instructions

## Scrape every urls

In [12]:
recipes_urls[:5]

['https://www.veganrecipeclub.org.uk/recipes/beetroot-dhal/',
 'https://www.veganrecipeclub.org.uk/recipes/avocado-wings-with-ranch-dip/',
 'https://www.veganrecipeclub.org.uk/recipes/black-forest-gateau-smoothie/',
 'https://www.veganrecipeclub.org.uk/recipes/super-fast-healthy-ice-cream/',
 'https://www.veganrecipeclub.org.uk/recipes/52-fast-day-spiced-tomato-dahl/']

In [34]:
%%time
rows = []
columns = ['title', 'preparation_time', 'cooking_time', 'total_time', 'style', 'type', 'servings', 'ingredients', 'instructions']
for i, url in enumerate(recipes_urls):
    time.sleep(5)
    row = []
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        sections = get_sections(soup)

        row.append(get_title(soup))
        row.append(get_preparation_time(soup, sections))
        row.append(get_cooking_time(soup, sections))
        row.append(get_total_time(soup, sections))
        row.append(get_cooking_style(soup))
        row.append(get_recipe_type(soup, sections))
        row.append(get_number_servings(soup, sections))
        row.append(get_ingredients(soup, sections))
        row.append(get_instructions(soup, sections))

        rows.append(row)
    except:
        print('link:', url, 'had an error')
    
    print("Progress:", round(100*i/len(recipes_urls),2),"%", end='\r')
    
df = pd.DataFrame(rows, columns = columns)

link: /author/justbloggs had an error
link: /login/ had an error
link: /author/vegan-north had an error
Wall time: 1h 17min 29s


In [35]:
df

,title,preparation_time,cooking_time,total_time,style,type,servings,ingredients,instructions
0,Beetroot Dhal,,0,60,Indian and Indian Subcontinent,,2-4,"[2 beetroot, chopped into small cubes , Drizzl...","[Preheat your oven to 180°C/350°F/Gas Mark 4.,..."
1,Avocado Wings with Ranch Dip,,0,20,,,4,"[500ml high-quality soy milk, 42g lemon juice,...",[Combine all the ingredients (except the oil) ...
2,Black Forest Gateau Smoothie,,0,5 minutes,,,2,"[160g/1 cup frozen berries of your choice, 1 v...",[Blend all ingredients until smooth.]
3,Super-fast Healthy Ice Cream,,0,5-7 minutes,,,4-6,"[90g/⅔ cup raw, unsalted Brazil nuts, 8-9 fres...","[Blend the Brazil nuts, dates, water, and vani..."
4,5:2 Fast Day – Spiced Tomato Dahl,,0,40 minutes,Indian and Indian Subcontinent,,1,"[2 squirts oil spray (2 cals), 80g red onion, ...",[]
...,...,...,...,...,...,...,...,...,...
688,Sweet Potato Gnocchi with Sage Cashew Sauce,,0,0,Italian,,unknown,"[500g sweet potatoes, 130g grade 00 flour or p...",[Bake the sweet potatoes in their skins until ...
689,"Chilli Hotpot with Black Beans, Lentils & Choc...",,0,40 minutes,"Mexican, Central and South American",,4-6,"[1 onion, finely diced, 3 cloves garlic, finel...","[Fry the onion until soft., Add the garlic and..."
690,Banana Cardamom & Coconut Biscuits,,0,20,,,24,"[125g dark rye flour , 150g jumbo porridge oat...",[Preheat your oven to 180ºC/350ºF/Gas Mark 4. ...
691,Martin Shaw’s Shepherdess Pie,,0,45 minutes,,,4-6,[Cooked potatoes for mashing plus olive oil or...,"[Pre-heat the oven to 375ºF/190ºC/Gas Mark 5.,..."


## Transform columns of dataframe

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             693 non-null    object
 1   preparation_time  693 non-null    object
 2   cooking_time      693 non-null    int64 
 3   total_time        693 non-null    object
 4   style             693 non-null    object
 5   type              693 non-null    object
 6   servings          693 non-null    object
 7   ingredients       693 non-null    object
 8   instructions      693 non-null    object
dtypes: int64(1), object(8)
memory usage: 48.9+ KB


In [37]:
# for col in ['preparation_time', 'cooking_time', 'total_time']:
#     df[col] = df[col].fillna(0)
#     df[col] = df[col].astype(str)
#     df[col] = df[col].str.extract('(\d+)', expand=False)
#     df[col] = df[col].astype(int)

In [38]:
df

,title,preparation_time,cooking_time,total_time,style,type,servings,ingredients,instructions
0,Beetroot Dhal,,0,60,Indian and Indian Subcontinent,,2-4,"[2 beetroot, chopped into small cubes , Drizzl...","[Preheat your oven to 180°C/350°F/Gas Mark 4.,..."
1,Avocado Wings with Ranch Dip,,0,20,,,4,"[500ml high-quality soy milk, 42g lemon juice,...",[Combine all the ingredients (except the oil) ...
2,Black Forest Gateau Smoothie,,0,5 minutes,,,2,"[160g/1 cup frozen berries of your choice, 1 v...",[Blend all ingredients until smooth.]
3,Super-fast Healthy Ice Cream,,0,5-7 minutes,,,4-6,"[90g/⅔ cup raw, unsalted Brazil nuts, 8-9 fres...","[Blend the Brazil nuts, dates, water, and vani..."
4,5:2 Fast Day – Spiced Tomato Dahl,,0,40 minutes,Indian and Indian Subcontinent,,1,"[2 squirts oil spray (2 cals), 80g red onion, ...",[]
...,...,...,...,...,...,...,...,...,...
688,Sweet Potato Gnocchi with Sage Cashew Sauce,,0,0,Italian,,unknown,"[500g sweet potatoes, 130g grade 00 flour or p...",[Bake the sweet potatoes in their skins until ...
689,"Chilli Hotpot with Black Beans, Lentils & Choc...",,0,40 minutes,"Mexican, Central and South American",,4-6,"[1 onion, finely diced, 3 cloves garlic, finel...","[Fry the onion until soft., Add the garlic and..."
690,Banana Cardamom & Coconut Biscuits,,0,20,,,24,"[125g dark rye flour , 150g jumbo porridge oat...",[Preheat your oven to 180ºC/350ºF/Gas Mark 4. ...
691,Martin Shaw’s Shepherdess Pie,,0,45 minutes,,,4-6,[Cooked potatoes for mashing plus olive oil or...,"[Pre-heat the oven to 375ºF/190ºC/Gas Mark 5.,..."


In [39]:
df.to_csv(DATA_PATH / "recipes.csv")